In [1]:
# Next Steps: 
# try the current configuration for hyperparameter tuning, and use minibatches

In [1]:
#imports

import tensorflow as tf
#import tensorflow_models as tfm
from tensorflow.keras import datasets, layers, models, optimizers, losses, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
#from tensorflow_models.vision.heads import RPNHead
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import cv2 as cv
import random
import tqdm as tqdm
import scipy
import keras_tuner as kt

In [ ]:
def fix_filename(filename):
    fil1, fil2 = filename.split('/')
    if 'daySequence1' in filename:
        return 'daySequence1/daySequence1/frames/' + fil2
    elif 'daySequence2' in filename:
        return 'daySequence2/daySequence2/frames/' + fil2
    elif 'dayTraining' in filename:
        clip, c = fil2.split("-", maxsplit=1)
        return 'dayTrain/dayTrain/' + clip + "/frames/" + fil2
    elif 'nightSequence1' in filename:
        return 'nightSequence1/nightSequence1/frames/' + fil2
    elif 'nightSequence2' in filename:
        return 'nightSequence2/nightSequence2/frames/' + fil2
    elif 'nightTraining' in filename:
        clip, c = fil2.split("-", maxsplit=1)
        return 'nightTrain/nightTrain/' + clip + "/frames/" + fil2

#import the box annotation data
df_box = pd.DataFrame()
for dirname, _, filenames in os.walk('C:/Users/akhpv/OneDrive/Documents/Machine_Learning_Practice/LISA_Dataset'):
    for filename in filenames:
        if "BOX.csv" in filename:
            add_to_df = pd.read_csv(os.path.join(dirname,filename), sep=';')
            add_to_df = add_to_df.drop(add_to_df.columns[-4:], axis=1)
            for i, row in add_to_df.iterrows():
                filename = fix_filename(add_to_df.loc[i,'Filename'])
                add_to_df.loc[i,'Filename'] = filename

                
            df_box = pd.concat([df_box, add_to_df], ignore_index=True)



df_box.to_csv('df_img.csv')

KeyboardInterrupt: 

In [ ]:
#if you're using 
df_box = pd.read_csv('df_img.csv')
df_box.columns

df_box = df_box.drop('Unnamed: 0', axis=1)

df_box.columns

Index(['Filename', 'Annotation tag', 'Upper left corner X',
       'Upper left corner Y', 'Lower right corner X', 'Lower right corner Y'],
      dtype='object')

In [ ]:
df_box

,Filename,Annotation tag,Upper left corner X,Upper left corner Y,Lower right corner X,Lower right corner Y
0,daySequence1/daySequence1/frames/daySequence1-...,stop,706,478,718,500
1,daySequence1/daySequence1/frames/daySequence1-...,stop,705,475,720,497
2,daySequence1/daySequence1/frames/daySequence1-...,stop,707,476,719,494
3,daySequence1/daySequence1/frames/daySequence1-...,stop,708,474,720,492
4,daySequence1/daySequence1/frames/daySequence1-...,stop,707,470,722,492
...,...,...,...,...,...,...
112365,nightTrain/nightTrain/nightClip1/frames/nightC...,go,872,17,958,143
112366,nightTrain/nightTrain/nightClip1/frames/nightC...,go,938,5,1028,131
112367,nightTrain/nightTrain/nightClip1/frames/nightC...,go,1006,6,1094,117
112368,nightTrain/nightTrain/nightClip1/frames/nightC...,go,1081,0,1171,110


In [ ]:
df_box['Annotation tag'].value_counts()

Annotation tag
go             48892
stop           44730
stopLeft       13048
warning         2669
goLeft          2476
warningLeft      350
goForward        205
Name: count, dtype: int64

In [ ]:
for tag in ['warning', 'goLeft', 'warningLeft', 'goForward']:
    df_box = df_box.drop(df_box[df_box['Annotation tag'] == tag].index)

# Count the occurrences of each value in the 'Category' column
category_counts = df_box['Annotation tag'].value_counts()

# Determine the minimum count of occurrences for each category
min_count = category_counts.min() 

# Initialize an empty DataFrame to store the sampled rows
sampled_df = pd.DataFrame(columns=df_box.columns)

# Sample an equal number of rows for each category
for category in category_counts.index:
    category_samples = df_box[df_box['Annotation tag'] == category].sample(min_count)
    sampled_df = pd.concat([sampled_df, category_samples], ignore_index=True)

sampled_df['Annotation tag'].value_counts()



Annotation tag
go          13048
stop        13048
stopLeft    13048
Name: count, dtype: int64

In [ ]:
sampled_df['label'] = 0


for i,row in sampled_df.iterrows():
    if sampled_df.loc[i, 'Annotation tag'] == 'stop':
        sampled_df.loc[i, 'label'] = 0
        
    elif sampled_df.loc[i, 'Annotation tag'] == 'stopLeft':
        sampled_df.loc[i, 'label'] = 1

    elif sampled_df.loc[i, 'Annotation tag'] == 'go':
        sampled_df.loc[i, 'label'] = 2

In [ ]:
sampled_df = sampled_df.sample(frac = 1, ignore_index = True)

In [ ]:
sampled_df.to_csv('sampled_df.csv')

In [2]:
#if you want to get the sampled_df automatically:
sampled_df = pd.read_csv('sampled_df.csv')

In [3]:
#split sample data into training and testing dataframe

split_point = int(len(sampled_df)*0.8)
sampled_df_train = sampled_df.loc[0:split_point]
sampled_df_test = sampled_df.loc[split_point: len(sampled_df)]

In [4]:
path = 'C:/Users/akhpv/OneDrive/Documents/Machine_Learning_Practice/LISA_Dataset'

train_datagen = ImageDataGenerator(validation_split=0.2)
test_datagen = ImageDataGenerator()
#maybe add validation, but I gotta learn what it is first

train_set = train_datagen.flow_from_dataframe(dataframe = sampled_df_train, x_col='Filename', y_col = 'Annotation tag', target_size=(192,256), subset="training", class_mode="sparse")
print('i')
val_set = train_datagen.flow_from_dataframe(dataframe = sampled_df_train, x_col='Filename', y_col = 'Annotation tag', target_size=(192,256), subset="validation", class_mode="sparse")
print('j')
test_set = test_datagen.flow_from_dataframe(dataframe = sampled_df_test, x_col='Filename', y_col = 'Annotation tag', target_size=(192,256), class_mode="sparse")

Found 25053 validated image filenames belonging to 3 classes.
i
Found 6263 validated image filenames belonging to 3 classes.
j
Found 7829 validated image filenames belonging to 3 classes.


In [5]:
#with OpenCV, we can resize each image in our dataset and then get the RGB values of each pixel
#This is an example of that, with an image to show how the image itself looks
#resized images will be of shape 64x64

def print_img(filename):
    img = cv.imread(filename)
    print(img.shape)
    resize = cv.resize(img, (256, 256))

    print(resize.shape)
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    resize = cv.cvtColor(resize, cv.COLOR_BGR2RGB)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    ax1.imshow(img)
    ax2.imshow(resize)



## Make Model and Train/Test It


## Hyperparameter Tuning Approach

In [6]:


def model_builder(hp):
  model = models.Sequential()

  #hyperparameter for number of layers
  #hyperparameter for filters in each layer
  #hyperparameter for window size in each conv2D layer

  hp_num_conv = 2

  hp_layers_list = []
  for i in range(hp_num_conv):
    hp_layer = hp.Choice(name = 'layer_'+str(i), values = [32, 64])
    hp_layers_list.append(hp_layer)


  for i in hp_layers_list:
    model.add(layers.Conv2D(filters = i, kernel_size = (3, 3), activation='relu', input_shape=(192,256, 3)))
    model.add(layers.MaxPooling2D((2, 2)))

  model.add(layers.Flatten())
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=128, max_value=512, step=128)
  model.add(layers.Dense(units=hp_units, activation='relu'))
  model.add(layers.Dense(3))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01 or 0.001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3])

  model.compile(optimizer=optimizers.Adam(learning_rate=hp_learning_rate),
                loss=losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [7]:
tuner_1 = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     factor=3,
                     project_name='hyperparam_tuning_results')

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)


INFO:tensorflow:Reloading Tuner from .\hyperparam_tuning_results\tuner0.json


In [8]:
physical_devices = tf.config.list_physical_devices('GPU')
print("Available GPU devices:", physical_devices)

Available GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [9]:
tuner_1.search_space_summary(extended=False)

Search space summary
Default search space size: 4
layer_0 (Choice)
{'default': 32, 'conditions': [], 'values': [32, 64], 'ordered': True}
layer_1 (Choice)
{'default': 32, 'conditions': [], 'values': [32, 64], 'ordered': True}
units (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 512, 'step': 128, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001], 'ordered': True}


In [10]:
with tf.device('/GPU:0'):
    tuner_1.search(train_set, batch_size = 32, validation_data=val_set, callbacks=[stop_early])

INFO:tensorflow:Oracle triggered exit


In [11]:


# Get the optimal hyperparameters
best_hps=tuner_1.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")



The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 512 and the optimal learning rate for the optimizer
is 0.001.



In [12]:
best_hps_dict = best_hps.values
print(best_hps.values)

{'layer_0': 32, 'layer_1': 64, 'units': 512, 'learning_rate': 0.001, 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 4, 'tuner/round': 0}


In [13]:
best_hps['layer_1']

64

In [14]:
final_model = models.Sequential()
final_model.add(layers.Conv2D(filters = best_hps['layer_0'], kernel_size = (3, 3), activation='relu', input_shape=(192,256, 3)))
final_model.add(layers.MaxPooling2D((2, 2)))
final_model.add(layers.Conv2D(filters = best_hps['layer_1'], kernel_size = (3, 3), activation='relu', input_shape=(192,256, 3)))
final_model.add(layers.MaxPooling2D((2, 2)))
final_model.add(layers.Flatten())
final_model.add(layers.Dense(units=best_hps['units'], activation='relu'))
final_model.add(layers.Dense(3))

final_model.compile(optimizer=optimizers.Adam(learning_rate=best_hps['learning_rate']),
                loss=losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

In [15]:
with tf.device('/GPU:0'):
    final_model.fit(train_set, epochs = 10, callbacks = stop_early, validation_data = test_set)

Epoch 1/10
783/783 [==============================] - 196s 247ms/step - loss: 26.6421 - accuracy: 0.7118 - val_loss: 0.5409 - val_accuracy: 0.7709
Epoch 2/10
783/783 [==============================] - 158s 201ms/step - loss: 0.4673 - accuracy: 0.7884 - val_loss: 0.5003 - val_accuracy: 0.7747
Epoch 3/10
783/783 [==============================] - 160s 204ms/step - loss: 0.4104 - accuracy: 0.8057 - val_loss: 0.5138 - val_accuracy: 0.7642
Epoch 4/10
783/783 [==============================] - 158s 202ms/step - loss: 0.3841 - accuracy: 0.8114 - val_loss: 0.5233 - val_accuracy: 0.7678
Epoch 5/10
783/783 [==============================] - 157s 200ms/step - loss: 0.3668 - accuracy: 0.8194 - val_loss: 0.5429 - val_accuracy: 0.7573
Epoch 6/10
783/783 [==============================] - 159s 203ms/step - loss: 0.3567 - accuracy: 0.8231 - val_loss: 0.5662 - val_accuracy: 0.7719
Epoch 7/10
783/783 [==============================] - 156s 199ms/step - loss: 0.3455 - accuracy: 0.8260 - val_loss: 0.5512 

In [17]:
plt.plot(final_model.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

TypeError: 'History' object is not subscriptable